The following code are based on: https://github.com/atong01/conditional-flow-matching/tree/main

#Calculate the FID score

The first step is to acces the shareable link below.

https://drive.google.com/drive/folders/1IIcsUvqVemkcv93kVy6do5_V0MRxhWZl?usp=sharing

Next step is to mount this file. To achieve this the following has to be done

1. Go to 'Shared with me' in Google Drive
3. Right click on the folder 'results' and click 'Organize > Add shortcut to drive'.
4. A pop-up window will apear, select 'MyDrive' then click on 'Add Shortcut'.
5. We are now able to mount to Google Drive with


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


The next step is to install the packages that are needed

In [ ]:
#python -m pip install --upgrade pip

!pip install clean-fid
!pip install torch
!pip install torchdiffeq
!pip install torchdyn
!pip install torchcfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 781.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.8/300.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.1/94.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.0/790.0 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 16.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 12.

#Import Libaries


In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import torch
from absl import app, flags
from cleanfid import fid
from torchdiffeq import odeint
from torchdyn.core import NeuralODE

from torchcfm.models.unet.unet import UNetModelWrapper

/usr/local/lib/python3.10/dist-packages/ot/backend.py:2998: UserWarning: To use TensorflowBackend, you need to activate the tensorflow numpy API. You can activate it by running: 
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
  register_backend(TensorflowBackend())


#Defining the parameters for the model

In [ ]:
# UNet
num_channel= 128 #base channel of UNet

# Building
parallel = False #multi gpu training
integration_steps = 100 #number of inference steps
integration_method = "dopri5" #integration method to use
step = 40000 #training steps
num_gen = 50000 #number of samples to generate
tol = 1e-5 #Integrator tolerance (absolute and relative)



#Define the model

In [ ]:

# Define the model
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

new_net = UNetModelWrapper(
    dim=(3, 32, 32),
    num_res_blocks=2,
    num_channels=num_channel,
    channel_mult=[1, 2, 2, 2],
    num_heads=4,
    num_head_channels=64,
    attention_resolutions="16",
    dropout=0.1,
).to(device)

#Load the model

In [ ]:
# Load the model
PATH_ot = f"/content/drive/My Drive/results/cifar10_weights_step_{step}_OT.pt" #This is the path for the OT model
PATH_vp = f"/content/drive/My Drive/results/cifar10_weights_step_{step}_Diffusion.pt" #This is the path for the VP model
print("path: ", PATH_ot) #For running the FID score for the Diffusion model change "PATH_ot" to "PATH_vp"
checkpoint = torch.load(PATH_ot, map_location=torch.device('cpu')) #For running the FID score for the Diffusion model change "PATH_ot" to "PATH_vp"
state_dict = checkpoint["ema_model"]

try:
    new_net.load_state_dict(state_dict)
except RuntimeError:
    from collections import OrderedDict

    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        new_state_dict[k[7:]] = v
    new_net.load_state_dict(new_state_dict)
new_net.eval()


path:  /content/drive/My Drive/results/cifar10_weights_step_40000_OT.pt


UNetModelWrapper(
  (time_embed): Sequential(
    (0): Linear(in_features=128, out_features=512, bias=True)
    (1): SiLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
  )
  (input_blocks): ModuleList(
    (0): TimestepEmbedSequential(
      (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1-2): 2 x TimestepEmbedSequential(
      (0): ResBlock(
        (in_layers): Sequential(
          (0): GroupNorm32(32, 128, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
        (h_upd): Identity()
        (x_upd): Identity()
        (emb_layers): Sequential(
          (0): SiLU()
          (1): Linear(in_features=512, out_features=128, bias=True)
        )
        (out_layers): Sequential(
          (0): GroupNorm32(32, 128, eps=1e-05, affine=True)
          (1): SiLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Conv2d(128, 128, kernel_size

###This step is for using the euler method instead of dopri5, however dopri5 should be used

In [ ]:
# Define the integration method if euler is used
if integration_method == "euler":
    node = NeuralODE(new_net, solver=integration_method)


def gen_1_img(unused_latent):
    with torch.no_grad():
        x = torch.randn(500, 3, 32, 32).to(device)
        if integration_method == "euler":
            print("Use method: ", integration_method)
            t_span = torch.linspace(0, 1, integration_steps + 1).to(device)
            traj = node.trajectory(x, t_span=t_span)
        else:
            print("Use method: ", integration_method)
            t_span = torch.linspace(0, 1, 2).to(device)
            traj = odeint(
                new_net, x, t_span, rtol=tol, atol=tol, method=integration_method
            )
    traj = traj[-1, :]  # .view([-1, 3, 32, 32]).clip(-1, 1)
    img = (traj * 127.5 + 128).clip(0, 255).to(torch.uint8)  # .permute(1, 2, 0)
    return img



## Calculate the FID

In [ ]:

print("Start computing FID")
score = fid.compute_fid(
    gen=gen_1_img,
    dataset_name="cifar10",
    batch_size=500,
    dataset_res=32,
    num_gen=num_gen,
    dataset_split="train",
    mode="legacy_tensorflow",
)
print()
print("FID has been computed")
# print()
# print("Total NFE: ", new_net.nfe)
print()
print("FID: ", score)

Start computing FID
compute FID of a model with cifar10-32 statistics
downloading statistics to /usr/local/lib/python3.10/dist-packages/cleanfid/stats/cifar10_legacy_tensorflow_train_32.npz


KeyboardInterrupt: ignored